In [1]:
# This is the streamlined version of continued.ipynp with just the code that we use in the end
from math import sqrt
import numpy as np

In [2]:
# This cell and the next provides the function to compute divisors
# We set up a table where each composite m maps to a pair of factors whose product is x
# With this we can quickly compute the prime factorization and all divisors
from collections import Counter

# Make table of least prime factors up to n
def least_prime_factor_table(n):
    u = list(range(n + 1))
    for i in range(2, int(sqrt(n)) + 1):
        if u[i] == i:
            for j in range(2 * i, n + 1, i):
                if u[j] == j:
                    u[j] = i
    return u


class Factorizer:
    def __init__(self, n = None):
        if n:
            self.m = least_prime_factor_table(n)

    
    def unsorted_prime_factors(s, k):
        if s.m[k] == k:
            return [k]
        else:
            u = s.m[k]
            v = k//u
            return [u] + s.unsorted_prime_factors(v)

    def prime_factors(s, k):
        result = []
        kk = k
        while(kk > 1):
            u = s.m[kk]
            result.append(u)
            kk = kk//u
        return result

    def divisors_from_prime_powers(s, uu):
        if len(uu) == 0:
            return [1]
        p, r = uu[0]
        dd = s.divisors_from_prime_powers(uu[1:])
        result = dd
        for _ in range(r):
            dd = [x * p for x in dd]
            result += dd
        return result

    def divisors(s, k):
        u = s.prime_factors(k)
        uu = list(Counter(u).items())
        return sorted(s.divisors_from_prime_powers(uu))


In [3]:
# A table up to 125 million will provide divisors up to length 20
f81 = Factorizer(10 * 10 ** 6)
divisors = f81.divisors


In [4]:
# We search for (A, B, C) with B^2 - 4AC = D and the roots x, x' of Ax^2 - Bx + C = 0 satisfy -1 < x < 0 < 1 < x' 
# We require a function to compute divisors as a global variable
def qirs(d):
    global divisors
    result = []
    if d % 4 == 2 or d % 4 == 3:
        return result
    sqrt_d = sqrt(d)
    for b in range(d % 2, int(sqrt(d) + 1), 2):       
        divs = divisors(int((d - b * b)/4))
        for a in divs:
            if b + sqrt_d > 2 * a and sqrt_d - b < 2 * a:
                result.append((b, a, d))
    return result

In [14]:
def rotl(x, k):
    return x[k:] + x[:k]

def min_rot(x):
    return min(rotl(x, k) for k in range(len(x)))

def min_rot2(x):
    return min(rotl(x, k) for k in range(0, len(x), 2))

In [15]:
# Gets the matrix with one fixed point in (-1, 0) and the other in (1, infinity)
def raw_matrix(trace, B, A, D):
    a = int((trace + B)/2) # (trace + B)/2 is already a integer, but python needs to know it 
    d = trace - a
    c = A
    C = int((B ** 2 - D)/(4 * A))
    b = -C
    return np.array([[a, b], [c, d]])

# Gets all the raw matrices for a given trace
def raw_matrix_seq(trace):
    a = qirs(trace ** 2 - 4)
    result = []
    for B, A, D in a:
        result.append(raw_matrix(trace, B, A, D))
    return result

# Makes pairs of the matrices along with their 'n', where the second fixed point is in (n, n+1)
def raw_matrix_and_n_seq(trace):
    D = trace ** 2 - 4
    sqrt_D = sqrt(D)
    a = qirs(D)
    result = []
    for B, A, D in a:
        n = int((B + sqrt_D)/(2 * A))
        result.append((raw_matrix(trace, B, A, D), n))
    return result
    
# Conjugates M to put into into the next acceptable form
# (Where acceptable means one fixed point in (-1, 0) and the other in (1, infinity))
def one_step_conj(M, n):
    U = np.array([[0, 1], [1, -n]])
    return conj0by1(M, U)

# Converts the 2x2 matrix into a tuple so we can look it up in a dictionary
def mat2tup(U):
    return tuple(U.reshape(4))

# computes the inverse of a matrix in GL_2(Z)
def int_inv(U):
    a, b, c, d = U.reshape(4)
    det = a * d - b * c
    return det * np.array([[d, -b],[-c, a]])

# Returns UTU^{-1}
def conj0by1(T, U):
    return np.matmul(np.matmul(U, T), int_inv(U))

# Creates all the cutting sequences for matrices with a given trace > 3
# Does so by repeated renormalizing the raw matrix and accumulating an L^n or R^n until the circle closes
def new_per_seqs(trace):
    raws = raw_matrix_and_n_seq(trace)
    step = {mat2tup(M): (mat2tup(one_step_conj(M, n)), n) for (M, n) in raws}
    used = set()
    seqs = []
    for t in step:
        if t in used:
            continue
        used.add(t)
        (M, n) = step[t]
        l = [n]
        while M != t:
            used.add(M)
            (M, n) = step[M]
            l.append(n)
        seqs.append(min_rot(l))
    return seqs            

In [16]:
new_per_seqs(5)

[[1, 3]]

In [17]:
new_per_seqs(12)

[[1, 10], [2, 5]]

In [34]:
new_per_seqs(20)

[[2, 9], [1, 1, 2, 3], [1, 18], [1, 1, 3, 2], [3, 6]]

In [49]:
import time
start = time.time()
aaa=guys_for_trace(1000)
end = time.time()
print(end-start)

0.03721308708190918


In [50]:
len(aaa)

216

In [35]:
def trace2length(t):
    return 2 * np.arccosh(t/2)

def length2trace(l):
    return 2 * np.cosh(l/2)

# The sequences that we actually want, from the "condensed forms" that take primitive roots in PGL_2(Z)
def actual_sequences(a, trace):
    result = []
    for _x in a:
        x = _x[:] # Need this to make x harmlessly mutable
        m = m_rep(x)
        t = np.trace(m)
        if t < trace:
            x_length = trace2length(t)
            desired_length = trace2length(trace)
            k = int(desired_length/x_length + 0.5)
            x *= k
        # Symmetrize by both R-L flip and time reversal, and add all four
        if len(x) % 2 == 0:
            r = list(reversed(x))
            result += [x, rotl(x, 1), r, rotl(r, 1)]
        # Expand condensed form, and then symmetrize by time reversal
        else:
            xx = x + x
            result += [xx, list(reversed(x))]
    return result

def reduce_and_remove_duplicates(seqs):
    u = {tuple(min_rot2(x)) for x in seqs}
    return list(u)
            

def guys_for_trace(t):
    seqs = new_per_seqs(t)
    seqs1 = actual_sequences(seqs, t)
    seqs2 = reduce_and_remove_duplicates(seqs1)
    return seqs2


In [42]:
guys_for_trace(100)

[(1, 7, 6, 1),
 (1, 1, 1, 32),
 (1, 1, 6, 7),
 (1, 2, 19, 1),
 (1, 2, 3, 9),
 (1, 19, 2, 1),
 (1, 1, 1, 2, 4, 2),
 (1, 4, 1, 15),
 (2, 3, 9, 1),
 (2, 49),
 (1, 98),
 (7, 14),
 (1, 9, 3, 2),
 (2, 1, 9, 3),
 (1, 1, 2, 4, 2, 1),
 (1, 2, 1, 3, 3, 1),
 (1, 1, 19, 2),
 (4, 1, 15, 1),
 (49, 2),
 (1, 6, 7, 1),
 (1, 1, 32, 1),
 (1, 1, 3, 3, 1, 2),
 (1, 1, 7, 6),
 (1, 1, 2, 19),
 (14, 7),
 (98, 1),
 (1, 3, 3, 1, 2, 1),
 (1, 1, 2, 1, 3, 3)]

In [20]:
# Turns an EVEN LENGTH sequence of R's and L's (given as a sequence of how many R's and L's)
# into a matrix
def cf2matrix(a):
    u = np.eye(2, dtype=np.int64)
    for x in a:
        u = np.matmul(u, [[x, 1], [1, 0]])
    return u

def m_rep(seq):
    if len(seq) % 2 == 0:
        return cf2matrix(seq)
    else:
        return cf2matrix(seq + seq)

def m_reps(trace):
    a = min_seqs(trace)
    return [m_rep(x) for x in a]

def prim_m_reps(trace):
    a = m_reps(trace)
    return [x for x in a if np.trace(x) == trace]

In [21]:
((int(2 * np.cosh(20/2)))**2 - 4)/4

121286168.0

In [38]:
partial_result = None
def per_seqs_for_range(a, b, start_with_partial_result = False):
    global partial_result
    print("Going from {} to {}".format(a, b))
    if start_with_partial_result and partial_result:
        aa, result = partial_result
    else:
        aa, result = a, []
    for i in range(a, b):
        if i % 100 == 0:
            print("Starting {} at {}".format(i, datetime.datetime.now()))
            partial_result = (i, result)
        result += guys_for_trace(i)
    return result

def per_seqs_for_length(l):
    # a = int(2 * np.cosh(l/2)) + 1
    a = 5
    b = int(2 * np.cosh((l+1)/2)) + 1
    return per_seqs_for_range(a, b)

In [39]:
aaa = per_seqs_for_length(7)

Going from 5 to 55


In [40]:
len(aaa)

427

In [177]:
seqs_by_length = [[]] * 20
for i in range(1, 20):
    seqs_by_length[i] = per_seqs_for_length(i)

In [220]:
[len(seqs_by_length[i]) for i in range(1, 20)]

[1,
 1,
 5,
 10,
 25,
 48,
 127,
 303,
 736,
 1768,
 4497,
 11114,
 27944,
 71067,
 180094,
 460635,
 1181266,
 3036122,
 7831535]

In [242]:
u = str(seqs_by_length)
with open('sequences20', 'w') as handle:
    handle.write(u)

In [248]:
!gzip sequences20